In [1]:
# Import Packages
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse
import concurrent.futures
import pandas as pd

In [2]:
# Maximum number of threads that will be spawned
MAX_THREADS = 50

In [3]:
#Check out the html behind the url being scraped
from bs4 import BeautifulSoup
import requests
url = 'https://www.imdb.com/search/title/?release_date=2020-01-02,2021-02-01&user_rating=4.0,10.0&languages=en&count=250&ref_=adv_prv'
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)
print(soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   Released between 2020-01-02 and 2021-02-01,
User Rating between 4 and 10,
English
(Sorted by Popularity Ascending) - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
  

In [4]:
#Set up the empty arrays for data to be sent
movie_title_arr = []
movie_year_arr = []
movie_runtime_arr = []
movie_genre_arr = []
movie_cert_arr = []
movie_synopsis_arr =[]
movie_stars_arr = []
movie_votes_arr = []
movie_rating_arr = []
movie_metascore_arr = []
image_url_arr  = []
image_id_arr = []

In [9]:
def getMovieTitle(header):
    try:
        return header[0].find("a").getText()
    except:
        return 'NA'

def getReleaseYear(header):
    try:
        return header[0].find("span",  {"class": "lister-item-year text-muted unbold"}).getText()
    except:
        return 'NA'

def getCert(muted_text):
    try:
        return muted_text.find("span",  {"class":  "certificate"}).getText()
    except:
        return 'NA'    
    
def getRuntime(muted_text):
    try:
        return muted_text.find("span",  {"class":  "runtime"}).getText()
    except:
        return 'NA'
    
def getGenre(muted_text):
    try:
        return muted_text.find("span",  {"class":  "genre"}).getText()
    except:
        return 'NA'

def getsynopsys(movie):
    try:
        return movie.find_all("p", {"class":  "text-muted"})[1].getText()
    except:
        return 'NA'

def getvotes(votesDiv):
    try:
        return votesDiv.get('data-value')
    except:
        return 'NA'
    
def getstars(movie):
    try:
        return movie.find("p", {"class":  ""}).getText()
    except:
        return 'NA'

def getRating(ratingDiv):
    try:
        return ratingDiv.find("div", {"class": "inline-block ratings-imdb-rating"}).getText()
    except:
        return 'NA'

def getMetascore(ratingDiv):
    try:
        return ratingDiv.find("div", {"class": "inline-block ratings-metascore"}).getText()
    except:
        return 'NA'

def getImage(image):
    try:
        return image.get('loadlate')
    except:
        return 'NA'

def getImageId(image):
    try:
        return image.get('data-tconst')
    except:
        return 'NA'

In [10]:
def main(imdb_url):
    response = requests.get(imdb_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Movie Name
    movies_list  = soup.find_all("div", {"class": "lister-item mode-advanced"})
    
    for movie in movies_list:
        header = movie.find_all("h3", {"class":  "lister-item-header"})
        muted_text = movie.find_all("p", {"class":  "text-muted"})[0]
        star = movie.find_all("p", {"class":  ""})[0]
        ratingDiv = movie.find("div", {"class": "ratings-bar"})
        votesDiv = movie.find_all("p", {"class":  "sort-num_votes-visible"})
        rating = ratingDiv.find("div", {"class": "inline-block ratings-imdb-rating"})
        imageDiv =  movie.find("div", {"class": "lister-item-image float-left"})
        image = imageDiv.find("img", "loadlate")
        
        #  Movie Title
        movie_title =  getMovieTitle(header)
        movie_title_arr.append(movie_title)
        
        #  Movie release year
        year = getReleaseYear(header)
        movie_year_arr.append(year)
        
        #  Movie certification
        cert = getCert(muted_text)
        movie_cert_arr.append(cert)
        
        #  Movie runtime
        runtime = getRuntime(muted_text)
        movie_runtime_arr.append(runtime)
        
        #  Genre  of movie
        genre = getGenre(muted_text)
        movie_genre_arr.append(genre)
        
        # Movie Synopsys
        synopsis = getsynopsys(movie)
        movie_synopsis_arr.append(synopsis)
        
        # Movie votes
        votes = getvotes(votesDiv)
        movie_votes_arr.append(votes)
        
        # Movie Stars
        stars = getstars(movie)
        movie_stars_arr.append(stars)
        
        # Movie Rating
        rating_num = getRating(ratingDiv)
        movie_rating_arr.append(rating_num)
        
        rating_meta = getMetascore(ratingDiv)
        movie_metascore_arr.append(rating_meta)
        
        #  Image attributes
        img_url = getImage(image)
        image_url_arr.append(img_url)
        
        image_id = image.get('data-tconst')
        image_id_arr.append(image_id)

In [11]:
# An array to store all the URL that are being queried
imageArr = []

# Maximum number of pages one wants to iterate over
MAX_PAGE =51

# Loop to generate all the URLS.
for i in range(0,MAX_PAGE):
    totalRecords = 0 if i==0 else (250*i)+1
    print(totalRecords)
    imdb_url = f'https://www.imdb.com/search/title/?release_date=2020-01-02,2021-02-01&user_rating=4.0,10.0&languages=en&count=250&start={totalRecords}&ref_=adv_nxt'
    imageArr.append(imdb_url)


0
251
501
751
1001
1251
1501
1751
2001
2251
2501
2751
3001
3251
3501
3751
4001
4251
4501
4751
5001
5251
5501
5751
6001
6251
6501
6751
7001
7251
7501
7751
8001
8251
8501
8751
9001
9251
9501
9751
10001
10251
10501
10751
11001
11251
11501
11751
12001
12251
12501


In [12]:
def download_stories(story_urls):
    threads = min(MAX_THREADS, len(story_urls))
    with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
        executor.map(main, story_urls)

In [13]:
# Call the download function with the array of URLS called imageArr
download_stories(imageArr)

# Attach all the data to the pandas dataframe. You can optionally write it to a CSV file as well
movieDf = pd.DataFrame({
    "Title": movie_title_arr,
    "Release_Year": movie_year_arr,
    "Certification": movie_cert_arr,
    "Runtime": movie_runtime_arr,
    "Genre": movie_genre_arr,
    "Synopsis": movie_synopsis_arr,
    "Stars": movie_stars_arr,
    "Rating": movie_rating_arr,
    "Votes": movie_votes_arr,
    "Metascore": movie_metascore_arr,
    "image_url": image_url_arr,
    "image_id": image_id_arr,
})

#print('--------- Download Complete CSV Formed --------')

# movie.to_csv('file.csv', index=False) : If you want to store the file.
movieDf.head()

#--------- Download Complete CSV Formed --------

,Title,Release_Year,Certification,Runtime,Genre,Synopsis,Stars,Rating,Votes,Metascore,image_url,image_id
0,Bridgerton,(2020– ),MA15+,60 min,"\nDrama, Romance","\nWealth, lust, and betrayal set against the b...","\n \n Stars:\nPhoebe Dynevor, \n...",\n\n7.3\n,NA,NA,https://m.media-amazon.com/images/M/MV5BNjk4MD...,tt8740790
1,Upload,(2020– ),MA15+,30 min,"\nComedy, Mystery, Sci-Fi",\nA man is able to choose his own afterlife af...,"\n \n Stars:\nRobbie Amell, \nAn...",\n\n8.0\n,NA,NA,https://m.media-amazon.com/images/M/MV5BYjc5OW...,tt7826376
2,CODA,(2021),M,111 min,"\nComedy, Drama, Music",\nAs a CODA (Child of Deaf Adults) Ruby is the...,\n Director:\nSian Heder\n| \n Stars:\nE...,\n\n8.1\n,NA,\n74 \n Metascore\n,https://m.media-amazon.com/images/M/MV5BYzkyNz...,tt10366460
3,Star Trek: Picard,(2020– ),MA15+,46 min,"\nAction, Adventure, Drama",\nFollow-up series to Star Trek: The Next Gene...,"\n \n Stars:\nPatrick Stewart, \...",\n\n7.5\n,NA,NA,https://m.media-amazon.com/images/M/MV5BMmMzNT...,tt8806524
4,Raised by Wolves,(2020– ),MA15+,NA,"\nDrama, Fantasy, Sci-Fi",\nAndroids are tasked with raising human child...,"\n \n Stars:\nTravis Fimmel, \nA...",\n\n7.5\n,NA,NA,https://m.media-amazon.com/images/M/MV5BNzk0ZT...,tt9170108


In [15]:
#Data cleaning to be complete here...

In [37]:
#Analysis of DF to be complete here...
len(movieDf.index)

12750